## 기본 합성곱 신경망 구현

In [34]:
import tensorflow as tf
import numpy as np

## 하이퍼 파라미터

In [35]:
EPOCHS = 2

## 모델 정의

In [36]:
class ConvNet(tf.keras.Model): #convolutional Neural Network
    def __init__(self):
        super(ConvNet, self).__init__()
        conv2d=tf.keras.layers.Conv2D
        maxpool=tf.keras.layers.MaxPool2D
        self.sequence=list()
        self.sequence.append(conv2d(16,(3,3),padding='same',activation='relu'))#28x28x16 출력
        self.sequence.append(conv2d(16,(3,3),padding='same',activation='relu'))
        self.sequence.append(maxpool((2,2))) #14x14x16
        self.sequence.append(conv2d(32,(3,3),padding='same',activation='relu'))#14x14x32 출력
        self.sequence.append(conv2d(32,(3,3),padding='same',activation='relu'))
        self.sequence.append(maxpool((2,2))) #7 x 7 x 32
        self.sequence.append(conv2d(64,(3,3),padding='same',activation='relu'))#7 x 7 x 64 출력
        self.sequence.append(conv2d(64,(3,3),padding='same',activation='relu'))
        self.sequence.append(tf.keras.layers.Flatten()) # 7x7x64를 길쭉하게 하나의 벡터로 만든다.  1568개의 feature vector
        self.sequence.append(tf.keras.layers.Dense(128,activation='relu'))
        self.sequence.append(tf.keras.layers.Dense(10,activation='softmax'))
    def call(self, x, training=False, mask=None): #시퀀스의 레이어들을 모두 순회하면서
        for layer in self.sequence: #입력을 출력으로 계속해서 연결해줘서 최종 출력까지 내준다.
            x=layer(x)
        return x

## 학습, 테스트 루프 정의

In [37]:
# Implement training loop
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images)

    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

## 데이터셋 준비  
keras에 제공하는 mninst dataset을 그대로 사용해 보겠습니다.  
mnist 데이터셋을 케라스 데이터 셋 API로 부터 받아와서 mnist 데이터를 x train xtest 로 받아주고, train과 test를 가능한 곱하기 채널이 붙은 형태로 바꾸어준다.  
처음 들어오는 x_train같은 경우는 데이터 샘플 개수에다가 28 x 28 이런 3차원 텐서형태로 되어있다. 하지만 이거를 28  끝에 1을 붙여주어야지된다. 만약 2차원이아니라 3차원까지 채널을 추가해야 컨벌루션 레이어에서 제대로 동작을 합니다. 


In [38]:
mnist=tf.keras.datasets.mnist
(x_train,y_train),(x_test,y_test)=mnist.load_data()
x_train,x_test=x_train/255.0,x_test/255.0

# x_train:(NUM_SAMPLE,28,28)->(NUM_SAMPLE,28,28,1) 끝에 1을 붙여주는 이유는 2차원이아닌 3차원의 채널까지 추가해줘야 1채널이라는 영상을 알려준다.
x_train=x_train[...,tf.newaxis].astype(np.float32) #...은 train에 있는 모든 axis를 표현한 것입니다. [:,:,:]와 동일합니다.
x_test=x_test[...,tf.newaxis].astype(np.float32)

train_ds=tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(10000).batch(32)
test_ds=tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(32)


## 학습 환경 정의
### 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [39]:
# Create model
model = ConvNet()

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## 학습 루프 동작

In [40]:
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)

    for test_images, test_labels in test_ds:
        test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.12366034835577011, Accuracy: 96.19499969482422, Test Loss: 0.09190717339515686, Test Accuracy: 97.1199951171875
Epoch 2, Loss: 0.04430900886654854, Accuracy: 98.65499877929688, Test Loss: 0.031078368425369263, Test Accuracy: 99.01000213623047
